In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import requests

In [317]:
player_dataset = pd.read_html("https://basketball.realgm.com/nba/players")[4]
player_dataset = player_dataset[["#", "Player", "Pos", "HT", "Age", "Current Team"]] #Keep necessary cols

player_dataset


,#,Player,Pos,HT,Age,Current Team
0,5,Precious Achiuwa,SF,203cm,23,Toronto Raptors
1,4,Steven Adams,C,211cm,29,Memphis Grizzlies
2,13,Bam Adebayo,C,208cm,25,Miami Heat
3,30,Ochai Agbaji,SF,196cm,23,Utah Jazz
4,7,Santi Aldama,C,211cm,22,Memphis Grizzlies
...,...,...,...,...,...,...
504,21,Thaddeus Young,F,203cm,34,Toronto Raptors
505,11,Trae Young,PG,185cm,24,Atlanta Hawks
506,77,Omer Yurtseven,C,211cm,24,Miami Heat
507,44,Cody Zeller,F,211cm,30,Miami Heat


In [318]:
url = "https://basketballnoise.com/what-are-the-6-nba-divisions/" #Need to use requests to get around 403 error
toRead = requests.get(url)
teams_dataset = pd.read_html(toRead.text)[0]
#FIXING SPELLING
teams_dataset[["NBA Team"]] = teams_dataset[["NBA Team"]].replace("Philidelphia 76rs", "Philadelphia 76ers")
teams_dataset[["NBA Team"]] = teams_dataset[["NBA Team"]].replace("Charotte Hornets", "Charlotte Hornets")
teams_dataset[["NBA Team"]] = teams_dataset[["NBA Team"]].replace("Detriot Pistons", "Detroit Pistons")
teams_dataset[["NBA Team"]] = teams_dataset[["NBA Team"]].replace("Milwaulke Bucks", "Milwaukee Bucks")
teams_dataset[["NBA Team"]] = teams_dataset[["NBA Team"]].replace("Minnestoa Timberwolves", "Minnesota Timberwolves")
teams_dataset[["NBA Team"]] = teams_dataset[["NBA Team"]].replace("Portland Trailblazers", "Portland Trail Blazers")
teams_dataset[["NBA Team"]] = teams_dataset[["NBA Team"]].replace("LA Clippers", "Los Angeles Clippers")
teams_dataset

,NBA Team,NBA Division,NBA Conference
0,Boston Celtics,Atlantic Division,Eastern Conference
1,Brooklyn Nets,Atlantic Division,Eastern Conference
2,New York Knicks,Atlantic Division,Eastern Conference
3,Philadelphia 76ers,Atlantic Division,Eastern Conference
4,Toronto Raptors,Atlantic Division,Eastern Conference
5,Chicago Bulls,Central Division,Eastern Conference
6,Cleveland Cavaliers,Central Division,Eastern Conference
7,Detroit Pistons,Central Division,Eastern Conference
8,Indiana Pacers,Central Division,Eastern Conference
9,Milwaukee Bucks,Central Division,Eastern Conference


In [319]:
abbrv_data = pd.read_html("https://en.wikipedia.org/wiki/Wikipedia:WikiProject_National_Basketball_Association/National_Basketball_Association_team_abbreviations", header=0)[0]
abbrv_data = abbrv_data.rename(columns={"Franchise": "NBA Team", "Abbreviation/ Acronym":"Abb"})
abbrv_data

,Abb,NBA Team
0,ATL,Atlanta Hawks
1,BKN,Brooklyn Nets
2,BOS,Boston Celtics
3,CHA,Charlotte Hornets
4,CHI,Chicago Bulls
5,CLE,Cleveland Cavaliers
6,DAL,Dallas Mavericks
7,DEN,Denver Nuggets
8,DET,Detroit Pistons
9,GSW,Golden State Warriors


In [368]:
teams_dataset = pd.merge(teams_dataset, abbrv_data)[["NBA Team", "NBA Division", "NBA Conference", "Abb"]]

'Sacramento Kings'

In [321]:
def dataSetFilter(playerName, playerColor, teamName, teamColor, 
    confName, confColor, divName, divColor, posName, posColor,heightName, heightColor, heightDirection, 
    ageValue, ageColor, ageDirection, numValue, numColor, numDirection, player_dataset):

    if (playerColor == "green"):
        player_dataset = player_dataset[player_dataset["Player"] == playerName]
        return player_dataset
    else:
        player_dataset = player_dataset[player_dataset["Player"] != playerName]

    actualTeam = teams_dataset.loc[teams_dataset["Abb"] == teamName]["NBA Team"] #Full Team Name
    if (teamColor == "green"):
        player_dataset = player_dataset[player_dataset["Current Team"] == actualTeam.values[0]]
    else:
        player_dataset = player_dataset[player_dataset["Current Team"] != actualTeam.values[0]] #Team Filter

    if (confColor == "green"):
        notAllowedTeams = teams_dataset.loc[teams_dataset["NBA Conference"] != confName]["NBA Team"]
    else:
        notAllowedTeams = teams_dataset.loc[teams_dataset["NBA Conference"] == confName]["NBA Team"]
     
    for teamName in notAllowedTeams:
        player_dataset = player_dataset[player_dataset["Current Team"] != teamName] #Keep removing not allowed teams
    
    if (divColor == "green"):
        notAllowedTeamsUsingDiv = teams_dataset.loc[teams_dataset["NBA Division"] != divName]["NBA Team"]
    else:
        notAllowedTeamsUsingDiv = teams_dataset.loc[teams_dataset["NBA Division"] == divName]["NBA Team"]

    for teamName in notAllowedTeamsUsingDiv:
        player_dataset = player_dataset[player_dataset["Current Team"] != teamName] #Keep removing not allowed division teams


    if ((posColor == "green") | (posColor == "yellow")):
        if (posName == "C"):
            player_dataset = player_dataset[player_dataset["Pos"] == posName]
        elif ((posName == "G") | (posName == "F-G") | (posName == "G-F")):
            player_dataset = player_dataset[(player_dataset["Pos"] == posName) | (player_dataset["Pos"] == "PG") | (player_dataset["Pos"] == "SG") 
                                            | (player_dataset["Pos"] == "SF")]
        elif (posName == "F"):
            player_dataset = player_dataset[(player_dataset["Pos"] == posName) | (player_dataset["Pos"] == "SF") | (player_dataset["Pos"] == "PF")]
        elif ((posName == "C-F") | (posName == "F-C")):
            player_dataset = player_dataset[(player_dataset["Pos"] == "PF") | (player_dataset["Pos"] == "C")]
        else:
            player_dataset = player_dataset

    else:
        player_dataset = player_dataset #The difference between POTEL's database and the one I imported makes it dangerous to remove anything... revist this


    if (heightColor == "green"):
        player_dataset = player_dataset[player_dataset["HT"] == heightName]
    elif (heightColor == "yellow"):
        if (heightDirection == "up"):
            if ((heightName != "5-10") | (heightName != "6-10") | (heightName != "5-11") | (heightName != "6-11")): 
                player_dataset = player_dataset[(player_dataset["HT"] == (heightName[0:2] + str(int(heightName[3]) + 1))) | 
                                                (player_dataset["HT"] == (heightName[0:2] + str(int(heightName[3]) + 2)))] #LMAO
        else:
            if ((heightName != "6-0") | (heightName != "7-0") | (heightName != "6-1") | (heightName != "7-1")): 
                player_dataset = player_dataset[(player_dataset["HT"] == (heightName[0:2] + str(int(heightName[3]) - 1))) | 
                                                (player_dataset["HT"] == (heightName[0:2] + str(int(heightName[3]) - 2)))]
    else:
        player_dataset = player_dataset[player_dataset["HT"] != heightName] #Cant use comparison operator cuz of format


    ageValue = int(ageValue)
    if (ageColor == "green"):
        player_dataset = player_dataset[player_dataset["Age"] == ageValue]
    elif (ageColor == "yellow"):
        if (ageDirection == "up"):
            player_dataset = player_dataset[(player_dataset["Age"] == ageValue + 1) | (player_dataset["Age"] == ageValue + 2)]
        else:
            player_dataset = player_dataset[(player_dataset["Age"] == ageValue - 1) | (player_dataset["Age"] == ageValue - 2)]
    else:
        if (ageDirection == "up"):
            player_dataset = player_dataset[player_dataset["Age"] > ageValue]
        else:
            player_dataset = player_dataset[player_dataset["Age"] < ageValue]

    numValue = int(numValue)
    if (numColor == "green"):
        player_dataset = player_dataset[player_dataset["#"] == numValue]
    elif (numColor == "yellow"):
        if (numDirection == "up"):
            player_dataset = player_dataset[(player_dataset["#"] == numValue + 1) | (player_dataset["#"] == numValue + 2)]
        else:
            player_dataset = player_dataset[(player_dataset["#"] == numValue - 1) | (player_dataset["#"] == numValue - 2)]
    else:
        if (numDirection == "up"):
            player_dataset = player_dataset[player_dataset["#"] > numValue]
        else:
            player_dataset = player_dataset[player_dataset["#"] < numValue]

    return player_dataset

In [384]:
def finalFunction(iterator, datasetOfPlayers):
    while True:
        if (iterator == 1):        
            driver = webdriver.Safari()
            driver.maximize_window()
            driver.get("https://poeltl.dunk.town") #Open up webpage

        if (iterator == 1):
            removePopup = driver.find_element_by_class_name("top-bar")
            removePopup.click() #Get rid of initial popup
        
        time.sleep(2)
        playerInputTextBox = driver.find_element_by_id("autosuggest")

        if (iterator == 1):
            playerInputTextBox.click()

        if (iterator == 1):
            playerInputTextBox.send_keys("Ben Simmons")
        else:
            playerInputTextBox.send_keys(datasetOfPlayers[["Player"]].sample().values[0][0])
            toPrint = datasetOfPlayers[["Player"]].sample().values[0][0] 
                         
        time.sleep(1.5)
        playerInputTextBox.send_keys(Keys.ENTER)

        if (iterator != 1):
            if (datasetOfPlayers.shape[0] == 1):
                print("Today's player was " + toPrint + "!")
                print("It took the solver " + str(iterator) + " guesses!")
                break

        # height = driver.find_element_by_xpath("//*[@id='__layout']/div/main/div[2]/div/div[2]/div[1]/div[6]/div/div[1]").text

        #print(height)

        def getPlayerInfo(index=iterator):
            playerCell = driver.find_element_by_xpath("//*[@id='__layout']/div/main/div[2]/div/div[2]/div["+str(index)+"]/div[1]")
            playerName = playerCell.text.strip('\n')
            playerName = playerName.strip()
            playerColor = playerCell.value_of_css_property("background-color")
            if (playerColor == 'rgba(0, 0, 0, 0)'):
                playerColor = "white"
            else:
                playerColor = "green"
            return playerName, playerColor

        def getTeamInfo(index=iterator):
            teamCell = driver.find_element_by_xpath("//*[@id='__layout']/div/main/div[2]/div/div[2]/div["+str(index)+"]/div[2]")
            teamName = teamCell.text.strip()
            teamColor = teamCell.value_of_css_property("background-color")
            if (teamColor == 'rgb(55, 190, 117)'):
                teamColor = "green"
            else:
                teamColor = "white"
            return teamName, teamColor

        def getConfInfo(index=iterator):
            confCell = driver.find_element_by_xpath("//*[@id='__layout']/div/main/div[2]/div/div[2]/div["+str(index)+"]/div[3]")
            confName = confCell.text.strip()
            confColor = confCell.value_of_css_property("background-color")

            if (confName == "West"):
                confName = "Western Conference"
            else:
                confName = "Eastern Conference"

            if (confColor == 'rgb(55, 190, 117)'):
                confColor = "green"
            else:
                confColor = "white"

            return confName, confColor

        def getDivInfo(index=iterator):
            divCell = driver.find_element_by_xpath("//*[@id='__layout']/div/main/div[2]/div/div[2]/div["+str(index)+"]/div[4]")
            divName = divCell.text.strip()
            divColor = divCell.value_of_css_property("background-color")

            if (divName == "Atl."):
                divName = "Atlantic Division"
            elif (divName == "Cen."):
                divName = "Central Division"
            elif (divName == "NW"):
                divName = "Northwest Division"
            elif (divName == "Pac."):
                divName = "Pacific Division"
            elif (divName == "SE"):
                divName = "Southeast Division"
            else:
                divName = "SouthWest Division"
            
            if (divColor == 'rgb(55, 190, 117)'):
                divColor = "green"
            else:
                divColor = "white"

            return divName, divColor

        def getPosInfo(index=iterator):
            posCell = driver.find_element_by_xpath("//*[@id='__layout']/div/main/div[2]/div/div[2]/div["+str(index)+"]/div[5]")
            posName = posCell.text.strip()
            posColor = posCell.value_of_css_property("background-color")

            if (posColor == 'rgb(55, 190, 117)'):
                posColor = "green"
            elif (posColor == 'rgb(234, 221, 101)'):
                posColor = "yellow"
            else:
                posColor = "white"

            return posName, posColor

        def heightFormatter(height):
            height = height.replace("'", "-")
            height = height.replace("\"", "")
            height = height.strip()
            return height

        def getHeightInfo(index=iterator):
            heightCell = driver.find_element_by_xpath("//*[@id='__layout']/div/main/div[2]/div/div[2]/div["+str(index)+"]/div[6]")
            heightName = heightFormatter(heightCell.text)
            heightColor = heightCell.value_of_css_property("background-color")

            if (heightColor == 'rgb(55, 190, 117)'):
                    heightColor = "green"
            elif (heightColor == 'rgb(234, 221, 101)'):
                    heightColor = "yellow"
            else:
                    heightColor = "white" 

            #Getting Arrow Data
            direction = "equal"
            if (heightColor != "green"):
                listOfHeightContent = heightName.split("\n")
                actualHeight = listOfHeightContent[0]
                direction = listOfHeightContent[2]
                direction = direction.strip()
                if (direction == "↓"):
                    direction = "down"
                elif (direction == "↑"):
                    direction = "up"

            return heightName, heightColor, direction

        def getAgeInfo(index=iterator):
            ageCell = driver.find_element_by_xpath("//*[@id='__layout']/div/main/div[2]/div/div[2]/div["+str(index)+"]/div[7]")
            ageValue = ageCell.text
            ageColor = ageCell.value_of_css_property("background-color")

            if (ageColor == 'rgb(55, 190, 117)'):
                    ageColor = "green"
            elif (ageColor == 'rgb(234, 221, 101)'):
                    ageColor = "yellow"
            else:
                    ageColor = "white" 

            direction = "equal"
            listOfAgeContent = ageValue.split("\n")
            actualAge = listOfAgeContent[1].strip()
            if (ageColor != "green"):
                direction = listOfAgeContent[3]
                direction = direction.strip()
                if (direction == "↓"):
                    direction = "down"
                elif (direction == "↑"):
                    direction = "up"

            return actualAge, ageColor, direction


        def getNumInfo(index=iterator):
            numCell = driver.find_element_by_xpath("//*[@id='__layout']/div/main/div[2]/div/div[2]/div["+str(index)+"]/div[8]")
            numValue = numCell.text
            numColor = numCell.value_of_css_property("background-color")

            if (numColor == 'rgb(55, 190, 117)'):
                    numColor = "green"
            elif (numColor == 'rgb(234, 221, 101)'):
                    numColor = "yellow"
            else:
                    numColor = "white" 

            direction = "equal"
            listofNumContent = numValue.split('\n')
            actualNum = listofNumContent[1].strip()
            if (numColor != "green"):
                direction = listofNumContent[3]
                direction = direction.strip()
                if (direction == "↓"):
                    direction = "down"
                elif (direction == "↑"):
                    direction = "up"

            return actualNum, numColor, direction


        def getRowElementsAndStore(index=iterator):
            playerName, playerColor = getPlayerInfo(index)
            teamName, teamColor = getTeamInfo(index)
            confName, confColor = getConfInfo(index)
            divName, divColor = getDivInfo(index)
            posName, posColor = getPosInfo(index)
            heightName, heightColor, heightDirection = getHeightInfo(index)
            ageValue, ageColor, ageDirection = getAgeInfo(index)
            numValue, numColor, numDirection = getNumInfo(index)
            return ((playerName, playerColor), (teamName, teamColor), 
            (confName, confColor), (divName, divColor), (posName, posColor),(heightName[0:3], heightColor, heightDirection), 
            (ageValue, ageColor, ageDirection), (numValue, numColor, numDirection))

            
        rowData = getRowElementsAndStore(iterator)

        playerName = rowData[0][0]
        playerColor = rowData[0][1]
        teamName = rowData[1][0]
        teamColor = rowData[1][1]
        confName = rowData[2][0]
        confColor = rowData[2][1]
        divName = rowData[3][0]
        divColor = rowData[3][1]
        posName = rowData[4][0]
        posColor = rowData[4][1]
        heightName = rowData[5][0]
        heightColor = rowData[5][1]
        heightDirection = rowData[5][2]
        ageValue = rowData[6][0]
        ageColor = rowData[6][1]
        ageDirection = rowData[6][2]
        numValue = rowData[7][0]
        numColor = rowData[7][1]
        numDirection = rowData[7][2]


        if (iterator == 1):
            datasetOfPlayers = dataSetFilter(playerName, playerColor, teamName, teamColor, 
                confName, confColor, divName, divColor, posName, posColor,heightName, heightColor, heightDirection, 
                ageValue, ageColor, ageDirection, numValue, numColor, numDirection, player_dataset=player_dataset)
        else:
            datasetOfPlayers = dataSetFilter(playerName, playerColor, teamName, teamColor, 
                confName, confColor, divName, divColor, posName, posColor,heightName, heightColor, heightDirection, 
                ageValue, ageColor, ageDirection, numValue, numColor, numDirection, player_dataset=datasetOfPlayers)
        iterator += 1

In [385]:
finalFunction(1, player_dataset)

Today's player was Luke Kennard!
It took the solver 2 guesses!
